In [1]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

glueContext = GlueContext(SparkContext.getOrCreate())

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
11,application_1632897281066_0012,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
customersDF = glueContext.create_dynamic_frame.from_catalog(
             database="johndb",
             table_name="customers")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
customersDF.toDF().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------------------+--------------------+-------------+---------+---------+----------------+---------------+
|customerid|        customername|               email|         city|  country|territory|contactfirstname|contactlastname|
+----------+--------------------+--------------------+-------------+---------+---------+----------------+---------------+
|         1|   Land of Toys Inc.|  gladys.rim@rim.org|          NYC|      USA|       NA|           James|           Butt|
|         2|  Reims Collectables|yuki_whobrey@aol.com|        Reims|   France|     EMEA|       Josephine|        Darakjy|
|         3|     Lyon Souveniers|fletcher.flosi@ya...|        Paris|   France|     EMEA|             Art|         Venere|
|         4|   Toys4GrownUps.com| bette_nicka@cox.net|     Pasadena|      USA|       NA|           Lenna|       Paprocki|
|         5|Corporate Gift Id...|     vinouye@aol.com|San Francisco|      USA|       NA|         Donette|         Foller|
|         6|Technics Sto

In [4]:
def addcolumns(rec):
    rec["contactfullname"] = {}
    rec["contactfullname"] = rec["contactfirstname"] + ' ' + rec["contactlastname"]
    return rec

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
customersDF =  Map.apply(customersDF, f = addcolumns)
customersDF.toDF().show();

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------------------+-------------+---------------+-----------------+---------+--------------------+----------------+----------+
|territory|        customername|         city|contactlastname|  contactfullname|  country|               email|contactfirstname|customerid|
+---------+--------------------+-------------+---------------+-----------------+---------+--------------------+----------------+----------+
|       NA|   Land of Toys Inc.|          NYC|           Butt|       James Butt|      USA|  gladys.rim@rim.org|           James|         1|
|     EMEA|  Reims Collectables|        Reims|        Darakjy|Josephine Darakjy|   France|yuki_whobrey@aol.com|       Josephine|         2|
|     EMEA|     Lyon Souveniers|        Paris|         Venere|       Art Venere|   France|fletcher.flosi@ya...|             Art|         3|
|       NA|   Toys4GrownUps.com|     Pasadena|       Paprocki|   Lenna Paprocki|      USA| bette_nicka@cox.net|           Lenna|         4|
|       NA|Corporate

In [6]:
customersDF = DropFields.apply(customersDF,["contactfirstname","contactlastname"])
customersDF.toDF().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------------------+-------------+-----------------+---------+--------------------+----------+
|territory|        customername|         city|  contactfullname|  country|               email|customerid|
+---------+--------------------+-------------+-----------------+---------+--------------------+----------+
|       NA|   Land of Toys Inc.|          NYC|       James Butt|      USA|  gladys.rim@rim.org|         1|
|     EMEA|  Reims Collectables|        Reims|Josephine Darakjy|   France|yuki_whobrey@aol.com|         2|
|     EMEA|     Lyon Souveniers|        Paris|       Art Venere|   France|fletcher.flosi@ya...|         3|
|       NA|   Toys4GrownUps.com|     Pasadena|   Lenna Paprocki|      USA| bette_nicka@cox.net|         4|
|       NA|Corporate Gift Id...|San Francisco|   Donette Foller|      USA|     vinouye@aol.com|         5|
|       NA|Technics Stores Inc.|   Burlingame|   Simona Morasca|      USA|  gladys.rim@rim.org|         6|
|     EMEA|Daedalus Designs ...|     

In [7]:
def parsedomain(rec):
    rec["domain"] = {}
    indx = rec["email"].index('@')
    rec["domain"] = rec["email"][indx + 1:]
    return rec

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
customersDF =  Map.apply(customersDF, f = parsedomain)
customersDF.toDF().show();

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
customersDF = customersDF.rename_field("territory","region")
customersDF.toDF().show();

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------------------+--------------------+-------------+---------+----------------+---------------+------+
|customerid|        customername|               email|         city|  country|contactfirstname|contactlastname|region|
+----------+--------------------+--------------------+-------------+---------+----------------+---------------+------+
|         1|   Land of Toys Inc.|  gladys.rim@rim.org|          NYC|      USA|           James|           Butt|    NA|
|         2|  Reims Collectables|yuki_whobrey@aol.com|        Reims|   France|       Josephine|        Darakjy|  EMEA|
|         3|     Lyon Souveniers|fletcher.flosi@ya...|        Paris|   France|             Art|         Venere|  EMEA|
|         4|   Toys4GrownUps.com| bette_nicka@cox.net|     Pasadena|      USA|           Lenna|       Paprocki|    NA|
|         5|Corporate Gift Id...|     vinouye@aol.com|San Francisco|      USA|         Donette|         Foller|    NA|
|         6|Technics Stores Inc.|  gladys.rim@ri